# Section I: Basic Array Operations

* Julia Matrix Operation is fast. To know how fast it is, you need a benchmark package. Type `]` and `add BenchmarkTools` in your REPL.
* To enter REPL, type `julia` in your terminal (path should be set correctly).
* To get help in REPL, type `?` to enter API mode.
* To run shell command in REPL, type `;` to enter shell mode.
* To input `≈`, type `\approx<TAB>`, see full list of unicode input of julia [here](https://docs.julialang.org/en/v1/manual/unicode-input/).
* `@test` means stop program and throw and exception whenever a clause returns `false`. "function" start with `@` character is called a [macro](https://docs.julialang.org/en/v1/manual/metaprogramming/).

In [143]:
using BenchmarkTools, Test

cnot = [1 0 0 0; 0 1 0 0; 0 0 0 1; 0 0 1 0]  # a matrix
b = [1, 1im, true, 0.4e2] # a vector
c = [1, 1im, 0.4e2, true] # a vector

# run a test
@test cnot*b ≈ c

# run a benchmark
res1 = @benchmark $cnot*$b

BenchmarkTools.Trial: 
  memory estimate:  144 bytes
  allocs estimate:  1
  --------------
  minimum time:     50.365 ns (0.00% GC)
  median time:      63.638 ns (0.00% GC)
  mean time:        84.382 ns (15.04% GC)
  maximum time:     3.667 μs (95.93% GC)
  --------------
  samples:          10000
  evals/sample:     987

# Can it be faster? Yes if the matrix is small and static!

For static array, we can avoid all allocations.

### **Challenge!**
Read the document of package `StaticArrays`
https://github.com/JuliaArrays/StaticArrays.jl
and show it is really fast!

In [32]:
# install and using the package
# sa = <some static matrix> 
# sb = <some static vector> 

using StaticArrays: SMatrix, SVector
scnot = SMatrix{4, 4}(cnot)
sb = SVector{4}(b)

@test scnot*sb ≈ c

using Statistics: median
res2 = @benchmark $scnot*$sb
@test median(res2).time < median(res1).time / 5  # at least 5 times faster

Test Passed

# Linear Algebra

[docs](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/)

### **Challenge!**
Try to figure out how to use get eigenvalues, singular values and qr and pass the test

In [51]:
using LinearAlgebra: eigen, svd, qr
# hint, here you probabily want to type `?` to get help in REPL.

# sv = <get the singular value of matrix cnot>
# ev = <get the eigenvalue of matrix cnot>
# q = <get the Q matrix from QR decomposition of cnot>
sv = svd(cnot).S
ev = eigen(cnot).values
q = qr(cnot).Q

@test sv ≈ ones(4)
@test ev ≈ [-1, 1, 1, 1]
@test q*q' ≈ I

Test Passed

# Large sparse matrix

[KrylovKit](https://github.com/Jutho/KrylovKit.jl) is a package for solving large sparse matrix.

### **Challenge!**
Get the lowest singular value of specific sparse matrix

In [79]:
using SparseArrays
sp = kron(SparseMatrixCSC(cnot), sparse(I, 100, 100))

# ev = <get lowest eigenvalue of sp>
using KrylovKit
ev = eigsolve(sp, 1, :SR)[1][1]

@test vals[1] ≈ minimum(eigen(sp |> Matrix).values)

Test Passed

# Tensor contraction

[TensorOperations](https://github.com/Jutho/TensorOperations.jl) is a high performance package for tensor contractions.


### **Challenge!**
Try to calculate the following contraction
$C_{lj} = A_{i,j,k}B_{i,k,l}$

In [55]:
A = randn(6, 10, 5)
B = randn(6, 5, 7)

# ] add TensorOperations, and using it
# @tensor C[a, b] := <specify contraction>

# using TensorOperations
# @tensor C[l,j] := A[i,j,k] * B[i,k,l]

@test C |> size == (7, 10)  # here `x |> f` is same as calling f(x).

Test Passed

# Save and load data

[DelimitedFiles](https://docs.julialang.org/en/v1/stdlib/DelimitedFiles/index.html) is the `txt` format save and load standard module.

[JLD2 and FileIO](https://github.com/simonster/JLD2.jl) uses HDF5 format to save and load, suited for large data file.


### **Challenge!**
Read out the saved data.

In [142]:
using DelimitedFiles

a = randn(Float64, 3,3)
writedlm("_test.dat", a)
# b = <read data from file `_test.dat`>
b = readdlm("_test.dat")
@test b ≈ a

# FileIO
# ] add FileIO
using FileIO
jldopen("_example.jld2", "w") do f
    f["A"] = a
end

b_jld2 = load("_example.jld2")["A"]
@test b_jld2 ≈ a

Test Passed

# `######################################################`
# Now, you can embrace the world of Notebooks!
# But not programmer's world, this is why we have an advanced session ;D
# `######################################################`

# Defining functions

We have [multiple dispatch](https://docs.julialang.org/en/v1/manual/functions/) + [type tree](https://docs.julialang.org/en/v1/manual/types/) instead of classes in object oriented design.

In [144]:
f(x) = x^2
function f(x::Vector{T}) where T
    append!(copy(x), x)
end
function f(x::Vector{T}) where T
    append!(copy(x), x)
end
# inplace vector version of f
function f!(x::Vector{T}) where T
    append!(x, x)
end
f(x::String) = x^2

@test f(3) == 9
@test f.([1,2,3,4,5]) == [1,4,9,16,25]  # broadcast
@test f("bili") == "bilibili"
@test f.(["bili", "dili"]) == ["bilibili", "dilidili"]

x = [1, 2, 3]
@test f(x) == [1,2,3,1,2,3]  #
y = copy(x)
@test (f!(y); y) == [1,2,3,1,2,3]  # inplace version of f

Test Passed

# Final Boss

Fix the code of simulated annealing, this is the code chanlledge of 2016 UCAS summer school.